## Imports

In [1]:
# Imports
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

## Scraping organization-oriented view
https://audit.ecogood.org/firmenauskunft-fvz/

#### URL creation

In [2]:
# Basic URL for information search in ECG database
url = 'https://audit.ecogood.org/firmenauskunft-fvz/?ser_orga=&ser_land=&ser_branche=&ser_ftevon=&ser_ftebis=&ser_ort=&ser_b2b=&ser_b2c=&ser_fvz_view=_md&ser_fvz_pagesize=50&ser_rep=&ser_standard=&ser_auditart='

# Base url for pages when browsing through the table
base_url = 'https://audit.ecogood.org/firmenauskunft-fvz/?ser_orga&ser_land&ser_branche&ser_ftevon&ser_ftebis&ser_ort&ser_b2b&ser_b2c&ser_fvz_view=_md&ser_fvz_pagesize=50&ser_rep&ser_standard&ser_auditart&frm-page-19795='

In [3]:
# Create list with URLs to scrape through (as max. 50 entries per page are displayed)

# Creating soup
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
print(f'Response status: {response.status_code}')
print()

# Finding out number of total entries
total_entries = int(soup.select('strong')[0].get_text())
print(f'Total entries in ECG database: {total_entries}')

# Finding out number of URLs (pages)
number_of_urls = int(total_entries/50)+1
print(f'Number of table pages: {number_of_urls}')
print()

# Create dynamic list
dynamic_url_list = [base_url + str(i) for i in range(1,number_of_urls+1)]
print(f'URL list created. Length of dynamic_url_list: {len(dynamic_url_list)}')

Response status: 200

Total entries in ECG database: 1267
Number of table pages: 26

URL list created. Length of dynamic_url_list: 26


#### First overview

In [4]:
# Checking soup
print(soup.prettify())

<!DOCTYPE html>
<html class="avada-html-layout-wide avada-html-header-position-top" lang="de-DE" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   Firmenauskunft (FVZ) – Interactive – AuditAdministration
  </title>
  <meta content="max-image-preview:large" name="robots"/>
  <link href="https://audit.ecogood.org/firmenauskunft-fvz/" hreflang="de" rel="alternate"/>
  <link href="https://audit.ecogood.org/en-firmenauskunft-fvz-2/?lang=en" hreflang="en" rel="alternate"/>
  <link href="https://audit.ecogood.org/it-firmenauskunft-fvz-2/?lang=it" hreflang="it" rel="alternate"/>
  <link href="https://audit.ecogood.org/informaciones-sobre-la-empresa/?lang=es" hreflang="es" rel="alternate"/>
  <link href="//s.w.org" rel="dns-prefetch"/>
  <link href="https://audit.ecogood

In [5]:
# Checking tables and their classes (turns out there are sub tables in the main table)
for table in soup.select('table')[0:6]: # More "RW_Aufträge" sub tables coming after
    print(table.get('id'))

RW_Firmenliste
RW_Aufträge
RW_Aufträge
RW_Aufträge
RW_Aufträge
RW_Aufträge


In [6]:
# Checking column titles
company_data = soup.select('table')[0] # 'RW_Firmenliste is the relevant table'
column_titles = []
for header in company_data.select('tr')[0]:
    if header.get_text().strip() != '':
        column_titles.append(header.get_text().strip())

print(f'Table length: {len(column_titles)}')
print(f'\nColumn names: {str(column_titles)[1:-1]}')

Table length: 11

Column names: 'ID', 'Organisation', 'Land', 'PLZ', 'Ort', 'Größe', 'Branche', 'Bilanzdetails…', 'BilanztypPeer-Audit…', 'gültig bis…', 'Dokumente…'


#### Scraping all pages of the table

In [7]:
# Creating empty lists and dictionary
ids = []
organizations = []
organization_links = []
countries = []
postal_codes = []
cities = []
company_sizes = []
economic_sectors = []
balance_details = []

# Scraping
i=0
for url in dynamic_url_list:
    # Preparations
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    company_data = soup.select('table')[0]
    
    # Scraping main data
    for row in company_data.select('tbody tr'):
        if row.get_text().strip()[0] != '…': # In the subtables, the rows begin with '…'
            if row.get_text().strip().split('\n')[0] == '92435': # Special treatment of this case
                ids.append(row.get_text().strip().split('\n')[0])
                organizations.append((row.get_text().strip().split('\n')[1] + ' ' + row.get_text().strip().split('\n')[2]))
                countries.append(row.get_text().strip().split('\n')[3])
                postal_codes.append(row.get_text().strip().split('\n')[4])
                cities.append(row.get_text().strip().split('\n')[5])
                company_sizes.append(row.get_text().strip().split('\n')[6])
                economic_sectors.append(row.get_text().strip().split('\n')[7])
                balance_details.append(row.get_text().strip().split('\n')[8:])

            else:
                ids.append(row.get_text().strip().split('\n')[0])
                organizations.append(row.get_text().strip().split('\n')[1])
                countries.append(row.get_text().strip().split('\n')[2])
                postal_codes.append(row.get_text().strip().split('\n')[3])
                cities.append(row.get_text().strip().split('\n')[4])
                company_sizes.append(row.get_text().strip().split('\n')[5])
                economic_sectors.append(row.get_text().strip().split('\n')[6])
                balance_details.append(row.get_text().strip().split('\n')[7:])
    
    for a in company_data.find_all('a'):
        if 'firmenauskunft' in a.get('href'):
            organization_links.append(a.get('href'))
        
    # Show status
    i += 1
    print(f'Page {i}/{len(dynamic_url_list)} scraped.')
        
# Check
print(f'\nAll pages scraped. Length: {len(ids)} entries.')

Page 1/26 scraped.
Page 2/26 scraped.
Page 3/26 scraped.
Page 4/26 scraped.
Page 5/26 scraped.
Page 6/26 scraped.
Page 7/26 scraped.
Page 8/26 scraped.
Page 9/26 scraped.
Page 10/26 scraped.
Page 11/26 scraped.
Page 12/26 scraped.
Page 13/26 scraped.
Page 14/26 scraped.
Page 15/26 scraped.
Page 16/26 scraped.
Page 17/26 scraped.
Page 18/26 scraped.
Page 19/26 scraped.
Page 20/26 scraped.
Page 21/26 scraped.
Page 22/26 scraped.
Page 23/26 scraped.
Page 24/26 scraped.
Page 25/26 scraped.
Page 26/26 scraped.

All pages scraped. Length: 1267 entries.


In [8]:
# Checking scraping results / Creating first basic dataframe
ecg_org_df = pd.DataFrame({'id': ids, 
                    'organization': organizations,
                    'organization_link': organization_links,
                    'country': countries, 
                    'postal_code': postal_codes, 
                    'city': cities, 
                    'company_size': company_sizes,
                    'economic_sector': economic_sectors,
                    'balance_details': balance_details,
                    })
print(f'Shape: {ecg_org_df.shape}\n')
print(f'Datatype: \n{ecg_org_df.dtypes}\n')
display(ecg_org_df)

Shape: (1267, 9)

Datatype: 
id                   object
organization         object
organization_link    object
country              object
postal_code          object
city                 object
company_size         object
economic_sector      object
balance_details      object
dtype: object



,id,organization,organization_link,country,postal_code,city,company_size,economic_sector,balance_details
0,14885,4e solutions GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,70794,Filderstadt,(2)3-10,Handel / Konsum,"[, , …6451, 4.1 StandardPeerevaluation, 30.03..."
1,14886,4plus5,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,89077,Ulm,(2)3-10,Baugewerbe / Architektur,"[, , …6452, M5.0 KompaktbilanzPeerevaluation, ..."
2,35275,A & K Engemann GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,34439,Willebadessen,(2)3-10,Nahrungsmittel / Land / Forstwirtschaft,"[, , …24837, M5.0 KompaktbilanzPeerevaluation]"
3,44668,A&P Steuerberatungsgesellschaft mbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,14480,Potsdam,(3)11-25,Beratung / Consulting,"[, , …44667, 4.0 StandardPeerevaluation, 31.1..."
4,29652,AAP-ARCHITEKTEN ZT-GMBH,https://audit.ecogood.org/firmenauskunft-2/?la...,AT,1080,Wien,(3)11-25,Baugewerbe / Architektur,"[, , …29651, 4.1 StandardDeskaudit, , Bericht..."
...,...,...,...,...,...,...,...,...,...
1262,92844,Zimmerei Diedrich – Die Gesundhausbauer GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,37434,Rüdershausen,(3)11-25,Baugewerbe / Architektur,"[, , …92843, M5.0 KompaktbilanzPeerevaluation,..."
1263,45156,zimmerwerkstatt Nicola Bannier und Michael Weber,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,29456,Hitzacker,(1)1-2,Handwerk,"[, , …45155, 4.1 StandardPeerevaluation, 28.0..."
1264,165659,ZORA Kinder- und Jugendhilfe gGmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,17489,Greifswald,(5)51-100,Gesundheitswesen / Soziales / Pflege,"[, , …165658, M5.0 VollbilanzBesuchsaudit]"
1265,14718,Zukunftswerk eG,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,82319,Starnberg,(2)3-10,Beratung / Consulting,"[, , …3335, M5.0 KompaktbilanzPeerevaluation, ..."


#### Data cleaning and initial transformations

In [9]:
# Checking if specific case (that has a linebreak within the organization name) was scraped correctly
ecg_org_df[ecg_org_df['id'] == '92435']

,id,organization,organization_link,country,postal_code,city,company_size,economic_sector,balance_details
574,92435,IMPRESUM GRAFICAS LITOLEMA S.L.,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,46006,Valencia,(8) Todas las tallas,Otros,"[, , …92434, M5.0 Balance completoPeerevaluation]"


In [10]:
# Remove leading and trailing whitespaces
ecg_org_df = ecg_org_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [11]:
# Define and check missing values
ecg_org_df = ecg_org_df.applymap(lambda x: None if x == '' else x)

print(f'Missing values: \n{ecg_org_df.isna().sum()}\n')

Missing values: 
id                    0
organization          0
organization_link     0
country               0
postal_code           6
city                  0
company_size         20
economic_sector       4
balance_details       0
dtype: int64



In [12]:
# id
ecg_org_df['id'] = ecg_org_df['id'].astype(int) # As type integer
if ecg_org_df.duplicated(subset='id').any() == False: # Duplicates?
    print('No id duplicates.')

No id duplicates.


In [13]:
# organization

# missing values
if ecg_org_df['organization'].isna().any() == False:
    print('No missing values.')
else:
    print('Missing values detected:')
    print(ecg_org_df[ecg_org_df['organization'].isna()])

# duplicates
if ecg_org_df.duplicated(subset='organization').any() == False:
    print('\nNo organization duplicates.')
else:
    print('\nDuplicates detected:')
    display(ecg_org_df[ecg_org_df.duplicated(subset='organization', keep=False)])

No missing values.

Duplicates detected:


,id,organization,organization_link,country,postal_code,city,company_size,economic_sector,balance_details
48,164050,Apple,https://audit.ecogood.org/firmenauskunft-2/?la...,FR,None,Minsk,Wallmart,Jack Nicholson,[Für 164050 liegen keine entsprechenden Anträ...
49,164351,Apple,https://audit.ecogood.org/firmenauskunft-2/?la...,GI,None,Minsk,Nokia,Will smiff D,[Für 164351 liegen keine entsprechenden Anträ...
121,30326,Biohof Lecker GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,83410,Laufen,(2)3-10,Nahrungsmittel / Land / Forstwirtschaft,"[, , …30325, M5.0 KompaktbilanzPeerevaluation,..."
122,82924,Biohof Lecker GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,83410,Laufen,(2)3-10,Handel / Konsum,"[, , …82923, M5.0 VollbilanzPeerevaluation, 3..."
136,40988,Bioland Lammertzhof – Fam. Hannen GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,41564,Kaarst,(2)3-10,Nahrungsmittel / Land / Forstwirtschaft,"[, , …40989, 4.1 StandardDeskaudit, 31.12.201..."
137,85214,Bioland Lammertzhof – Fam. Hannen GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,41564,Kaarst,(5)51-100,Nahrungsmittel / Land / Forstwirtschaft,"[, , …85213, M5.0 KompaktbilanzPeerevaluation,..."
187,40892,camera obscura GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,10707,Berlin,(1)1-2,Medien,"[, , …40891, M5.0 KompaktbilanzPeerevaluation,..."
188,124487,camera obscura GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,10707,Berlin,(2)3-10,Marketing / Werbung / PR,"[, , …124486, M5.0 KompaktbilanzPeerevaluation..."
232,30962,corsus corporate sustainability GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,22765,Hamburg,(2)3-10,Beratung / Consulting,"[, , …30961, M5.0 KompaktbilanzDeskaudit, 30...."
233,153840,corsus corporate sustainability GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,20459,Hamburg,(10)2501-5000,Beratung / Consulting,"[, , …153839, M5.0 KompaktbilanzBesuchsaudit]"


In [14]:
# country
ecg_org_df = ecg_org_df.rename(columns={'country':'country_code'})

country_dictionary = {
    'AT': 'Austria',
    'BE': 'Belgium',
    'CH': 'Switzerland',
    'DE': 'Germany',
    'DK': 'Denmark',
    'ES': 'Spain',
    'HR': 'Croatia',
    'IT': 'Italy',
    'LU': 'Luxembourg',
    'NL': 'Netherlands',
    'PL': 'Poland',
    'PT': 'Portugal',
    'SE': 'Sweden',
    'UK': 'United Kingdom',
    'US': 'United States',
    'UR': 'Uruguay'
    }
ecg_org_df['country_name'] = ecg_org_df['country_code'].map(country_dictionary)

if ecg_org_df['country_name'].isna().any() == True:
    new_country_codes_df = ecg_org_df[ecg_org_df['country_name'].isna()][['id','country_code', 'city']]
    new_country_codes_summary = ecg_org_df[ecg_org_df['country_name'].isna()]['country_code'].value_counts()
    new_country_codes_count = len(new_country_codes_summary)
    print(f'{new_country_codes_count} new country code(s) found. Please review. \nCode(s) & count:')
    print(f'{new_country_codes_summary.to_string()}')
    print(f'\n{new_country_codes_df}')

else:
    print(f'Country codes of {len(country_dictionary)} countries mapped successfully.')

3 new country code(s) found. Please review. 
Code(s) & count:
FR    1
GI    1
NO    1

         id country_code   city
48   164050           FR  Minsk
49   164351           GI  Minsk
383  163006           NO  Minsk


In [15]:
# postal_code
if ecg_org_df['postal_code'].isna().any() == False:
    print('No missing postal codes.')

else:
    print('Missing postal codes before manual inserts:')
    print(ecg_org_df[ecg_org_df['postal_code'].isna()][['id', 'organization', 'country_code', 'postal_code', 'city']])

    ecg_org_df.loc[ecg_org_df['id'] == 153690, 'postal_code'] = '12400' # Known missing replacement

    print('\nMissing postal codes after manual inserts:')
    print(ecg_org_df[ecg_org_df['postal_code'].isna()][['id', 'organization', 'country_code', 'postal_code', 'city']])

Missing postal codes before manual inserts:
         id                organization country_code postal_code        city
22   164090                  AliExpress           ES        None       Minsk
48   164050                       Apple           FR        None       Minsk
49   164351                       Apple           GI        None       Minsk
383  163006                         FBI           NO        None       Minsk
463  165419                      Google           DE        None       Minsk
672  153690  LABORATORIO ECOTECH S.R.L.           UR        None  Montevideo

Missing postal codes after manual inserts:
         id organization country_code postal_code   city
22   164090   AliExpress           ES        None  Minsk
48   164050        Apple           FR        None  Minsk
49   164351        Apple           GI        None  Minsk
383  163006          FBI           NO        None  Minsk
463  165419       Google           DE        None  Minsk


In [16]:
# city
# Temporarily turn missing values into empty strings
ecg_org_df['city'] = ecg_org_df['city'].fillna('')

# Check for commas
if ecg_org_df['city'].str.contains(',').any() == False:
    print('No cases of commas in city column.')
else:
    print('Cases of commas in city column detected:')
    print(ecg_org_df[ecg_org_df['city'].str.contains(',')][['id', 'country_code', 'postal_code', 'city']])
    
# Check for digits
if ecg_org_df['city'].str.contains('\d').any() == False:
    print('\nNo cases of digits in city column.')
else:
    print('\nCases of digits in city column detected:')
    print(ecg_org_df[ecg_org_df['city'].str.contains('\d')][['id', 'country_code', 'postal_code', 'city']]) # works like this as well

# Check for underscores
if ecg_org_df['city'].str.contains('_').any() == False:
    print('\nNo cases of underscores in city column.')
else:
    print('\nCases of underscores in city column detected:')
    print(ecg_org_df[ecg_org_df['city'].str.contains('_')][['id', 'country_code', 'postal_code', 'city']]) # Anything not a letter, number, underscore

# Correcting known cases by hand
city_dictionary = {
    'Gent, Belgien': 'Gent',
    'Ettingen, Basel-Landschaft (Schweiz)': 'Ettingen (Basel-Landschaft)',
    '52074 Aachen': 'Aachen',
    'NL1 City': None
    }

ecg_org_df['city'] = ecg_org_df['city'].apply(lambda x: city_dictionary.get(x,x))

# Define missings as missing values again
ecg_org_df['city'] = ecg_org_df['city'].apply(lambda x: None if x == '' else x)

Cases of commas in city column detected:
         id country_code postal_code                                  city
627   14659           BE        9040                         Gent, Belgien
930  124736           CH        4107  Ettingen, Basel-Landschaft (Schweiz)

Cases of digits in city column detected:
         id country_code postal_code          city
15   132863           DE       52074  52074 Aachen
825   58436           NL       12345      NL1 City

No cases of underscores in city column.


In [17]:
# company_size
print('Company size value counts:')
print(ecg_org_df['company_size'].value_counts())

employees_dictionary = {
    '(1)1-2': '1-2',
    '(2)3-10': '3-10',
    '(3)11-25': '11-25',
    '(4)26-50': '26-50',
    '(5)51-100': '51-100',
    '(6)101-250': '101-250',
    '(7)>250': '>250',
    '(7)250-': '>250',
    '(8)501-1000': '501-1000',
    '(7)251-500': '251-500',
    '(10)2501-5000': '2501-5000',
    '(11)5001-10000': '5001-10000',
    '(9)1001-2500': '1001-2500',
    '(8) alle Größen': None,
    '(8) Todas las tallas': None,
    '': None
    }

company_size_category_dictionary = {
    '(1)1-2': 'Micro (≤10)',
    '(2)3-10': 'Micro (≤10)',
    '(3)11-25': 'Small (≤50)',
    '(4)26-50': 'Small (≤50)',
    '(5)51-100': 'Mid (≤250)',
    '(6)101-250': 'Mid (≤250)',
    '(7)>250': 'Large (>250)',
    '(7)250-': 'Large (>250)',
    '(8)501-1000': 'Large (>250)',
    '(7)251-500': 'Large (>250)',
    '(10)2501-5000': 'Large (>250)',
    '(11)5001-10000': 'Large (>250)',
    '(9)1001-2500': 'Large (>250)',
    '(8) alle Größen': None,
    '(8) Todas las tallas': None,
    '': None
    }

ecg_org_df['company_size_category'] = ecg_org_df['company_size'].map(company_size_category_dictionary)
print('\nCompany size categories:')
print(ecg_org_df['company_size_category'].value_counts().to_string())

ecg_org_df['employees'] = ecg_org_df['company_size'].map(employees_dictionary)
print('\nEmployees:')
print(ecg_org_df['employees'].value_counts().to_string())

missing_company_sizes = int(ecg_org_df['company_size_category'].isna().sum())
print(f'\n{missing_company_sizes} entries are missing information about company size.')

Company size value counts:
(1)1-2                  395
(2)3-10                 328
(3)11-25                156
(4)26-50                121
(5)51-100                75
(6)101-250               59
(8) Todas las tallas     57
(7)>250                  25
(7)250-                  12
(8)501-1000               5
(7)251-500                3
(10)2501-5000             2
(11)5001-10000            2
Wallmart                  2
(8) alle Größen           1
(9)1001-2500              1
AliExpress                1
Apple                     1
Nokia                     1
Name: company_size, dtype: int64

Company size categories:
Micro (≤10)     723
Small (≤50)     277
Mid (≤250)      134
Large (>250)     50

Employees:
1-2           395
3-10          328
11-25         156
26-50         121
51-100         75
101-250        59
>250           37
501-1000        5
251-500         3
2501-5000       2
5001-10000      2
1001-2500       1

83 entries are missing information about company size.


In [18]:
# economic_sector

# First check
print('Initial check:')
print(f"{len(ecg_org_df['economic_sector'].value_counts())} unique sectors.\n")

unique_ordered_sectors = ecg_org_df['economic_sector'].sort_values(ascending=True)
unique_ordered_sectors = unique_ordered_sectors.drop_duplicates().reset_index(drop=True)

print('Unique values (first 10):')
print(unique_ordered_sectors.head(10))

Initial check:
72 unique sectors.

Unique values (first 10):
0                                     Abfallentsorgung
1                                  Advice / Consulting
2                        Art / Culture / Entertainment
3                                        Assicurazione
4    Assistenza sanitaria / Servizi sociali / Infer...
5                      Automobil / Automobilzulieferer
6                                               Banken
7                             Baugewerbe / Architektur
8                                             Beratung
9                                Beratung / Consulting
Name: economic_sector, dtype: object


In [19]:
sector_dictionary_EN = {
    'Abfallentsorgung': 'Waste Disposal',
    'Advice / Consulting': 'Consulting',
    'Art / Culture / Entertainment': 'Art / Culture / Entertainment',
    'Assicurazione': 'Waste Disposal',
    'Assistenza sanitaria / Servizi sociali / Infermieristica': 'Health / Social Affairs / Nursing',
    'Automobil / Automobilzulieferer': 'Automotive / Automotive supplier',
    'Banken': 'Banking',
    'Baugewerbe / Architektur': 'Construction / Architecture',
    'Beratung': 'Consulting',
    'Beratung / Consulting': 'Consulting',
    'Bertatung': 'Consulting',
    'Bildung / Universität / FH / Schulen': 'Education / University / Polytechnic / Schools',
    'Chemie': 'Chemistry',
    'Cibo / Agricoltura / Silvicoltura': 'Nutrition / Agriculture / Forestry',
    'Commercio / Consumo': 'Trade / Consumption',
    'Consigli / Consulenza': 'Consulting',
    'Construction / Architecture': 'Construction / Architecture',
    'Dienstleistung': 'Services',
    'Druck / Papier / Verpackung': 'Printing / Paper / Packaging',
    'EDP / IT': 'EDP / IT',
    'EDV / IT': 'EDP / IT',
    'Education / University / FH / Schools': 'Education / University / Polytechnic / Schools',
    'Electrical / Electronics': 'Electrical / Electronics',
    'Elektro / Elektronik': 'Electrical / Electronics',
    'Energiewirtschaft': 'Energy',
    'Finanzen': 'Finance',
    'Forschung / Entwicklung / Wissenschaft': 'Research / Development / Science',
    'Gesundheitswesen / Soziales / Pflege': 'Health / Social Affairs / Nursing',
    'Handel / Konsum': 'Trade / Consumption',
    'Handwerk': 'Craft',
    'Immobilien / Facility Management': 'Real Estate / Facility Management',
    'Industrie': 'Industry',
    'Industry': 'Industry',
    'Internet / Multimedia': 'Internet / Multimedia',
    'Kunst / Kultur / Unterhaltung': 'Art / Culture / Entertainment',
    'Marketing / Werbung / PR': 'Marketing / Avertising / PR',
    'Marktforschung': 'Market Research',
    'Maschinen / Anlagenbau': 'Machinery & Plant Engineering',
    'Media': 'Media',
    'Medien': 'Media',
    'Medizin / Pharma': 'Pharma',
    'Medizintechnik': 'Medical Engineering',
    'Nahrungsmittel / Land /  Forstwirtschaft': 'Nutrition / Agriculture / Forestry',
    'Nahrungsmittel / Land / Forstwirtschaft': 'Nutrition / Agriculture / Forestry',
    'Other branches': 'Other',
    'Otros': 'Other',
    'Personalwesen / Personalbeschaffung': 'HR',
    'Seminar / Messeanbieter': 'Seminar / Fair Provider',
    'Seminario / Fornitori di fiere': 'Seminar / Fair Provider',
    'Software · Branding · SEO · Gemeinwohl': 'EDP / IT',
    'Sonstige Branchen': 'Other',
    'Sport / Fitness / Beauty': 'Sports / Fitness / Beauty',
    'Steuerberatung / Wirtschaftsprüfung': 'Fiscal Advice / Auditing',
    'Telecommunication': 'Telecommunication',
    'Telekommunikation': 'Telecommunication',
    'Textilbranche': 'Textile',
    'Tourism / Hotel / Gastronomy': 'Tourism / Hotel / Catering',
    'Tourismus / Hotel / Gastronomie': 'Tourism / Hotel / Catering',
    'Tourismus / Hotel / Gastronomie/ Lebensmittel': 'Tourism / Hotel / Catering',
    'Turismo / Hotel / Gastronomia': 'Tourism / Hotel / Catering',
    'Vereine': 'Associations / Societies',
    'Verkehr / Transport / Logistik': 'Transport / Logistics',
    'Versicherung': 'Insurance',
    'sonstiges': 'Other',
    'Öffentliche Verwaltung': 'Public Administration',
    'Wirtschaftsförderung': 'Economic Promotion',
    '': None
    }

sector_dictionary_DE = {
    'Abfallentsorgung': 'Abfallentsorgung',
    'Advice / Consulting': 'Beratung / Consulting',
    'Art / Culture / Entertainment': 'Kunst / Kultur / Unterhaltung',
    'Assicurazione': 'Abfallentsorgung',
    'Assistenza sanitaria / Servizi sociali / Infermieristica': 'Gesundheitswesen / Soziales / Pflege',
    'Automobil / Automobilzulieferer': 'Automobil / Automobilzulieferer',
    'Banken': 'Banken',
    'Baugewerbe / Architektur': 'Baugewerbe / Architektur',
    'Beratung': 'Beratung / Consulting',
    'Beratung / Consulting': 'Beratung / Consulting',
    'Bertatung': 'Beratung / Consulting',
    'Bildung / Universität / FH / Schulen': 'Bildung / Universität / FH / Schulen',
    'Chemie': 'Chemie',
    'Cibo / Agricoltura / Silvicoltura': 'Nahrungsmittel / Land /  Forstwirtschaft',
    'Commercio / Consumo': 'Handel / Konsum',
    'Consigli / Consulenza': 'Beratung / Consulting',
    'Construction / Architecture': 'Baugewerbe / Architektur',
    'Dienstleistung': 'Dienstleistung',
    'Druck / Papier / Verpackung': 'Druck / Papier / Verpackung',
    'EDP / IT': 'EDV / IT',
    'EDV / IT': 'EDV / IT',
    'Education / University / FH / Schools': 'Bildung / Universität / FH / Schulen',
    'Electrical / Electronics': 'Elektro / Elektronik',
    'Elektro / Elektronik': 'Elektro / Elektronik',
    'Energiewirtschaft': 'Energiewirtschaft',
    'Finanzen': 'Finanzen',
    'Forschung / Entwicklung / Wissenschaft': 'Forschung / Entwicklung / Wissenschaft',
    'Gesundheitswesen / Soziales / Pflege': 'Gesundheitswesen / Soziales / Pflege',
    'Handel / Konsum': 'Handel / Konsum',
    'Handwerk': 'Handwerk',
    'Immobilien / Facility Management': 'Immobilien / Facility Management',
    'Industrie': 'Industrie',
    'Industry': 'Industrie',
    'Internet / Multimedia': 'Internet / Multimedia',
    'Kunst / Kultur / Unterhaltung': 'Kunst / Kultur / Unterhaltung',
    'Marketing / Werbung / PR': 'Marketing / Werbung / PR',
    'Marktforschung': 'Marktforschung',
    'Maschinen / Anlagenbau': 'Maschinen- & Anlagenbau',
    'Media': 'Medien',
    'Medien': 'Medien',
    'Medizin / Pharma': 'Medizin / Pharma',
    'Medizintechnik': 'Medizintechnik',
    'Nahrungsmittel / Land /  Forstwirtschaft': 'Nahrungsmittel / Land /  Forstwirtschaft',
    'Nahrungsmittel / Land / Forstwirtschaft': 'Nahrungsmittel / Land /  Forstwirtschaft',
    'Other branches': 'Sonstige',
    'Otros': 'Sonstige',
    'Personalwesen / Personalbeschaffung': 'Personalwesen / Personalbeschaffung',
    'Seminar / Messeanbieter': 'Seminar- / Messeanbieter',
    'Seminario / Fornitori di fiere': 'Seminar- / Messeanbieter',
    'Software · Branding · SEO · Gemeinwohl': 'EDV / IT',
    'Sonstige Branchen': 'Sonstige',
    'Sport / Fitness / Beauty': 'Sport / Fitness / Beauty',
    'Steuerberatung / Wirtschaftsprüfung': 'Steuerberatung / Wirtschaftsprüfung',
    'Telecommunication': 'Telekommunikation',
    'Telekommunikation': 'Telekommunikation',
    'Textilbranche': 'Textilbranche',
    'Tourism / Hotel / Gastronomy': 'Tourismus / Hotel / Gastronomie',
    'Tourismus / Hotel / Gastronomie': 'Tourismus / Hotel / Gastronomie',
    'Tourismus / Hotel / Gastronomie/ Lebensmittel': 'Tourismus / Hotel / Gastronomie',
    'Turismo / Hotel / Gastronomia': 'Tourismus / Hotel / Gastronomie',
    'Vereine': 'Vereine',
    'Verkehr / Transport / Logistik': 'Verkehr / Transport / Logistik',
    'Versicherung': 'Versicherung',
    'sonstiges': 'Sonstige',
    'Öffentliche Verwaltung': 'Öffentliche Verwaltung',
    'Wirtschaftsförderung': 'Wirtschaftsförderung', 
    '': None
    }

ecg_org_df['economic_sector_EN'] = ecg_org_df['economic_sector'].map(sector_dictionary_EN)
ecg_org_df['economic_sector_DE'] = ecg_org_df['economic_sector'].map(sector_dictionary_DE)

In [20]:
# Check EN
print('English:\n')
print(f"Sum of missing and unassigned values: {ecg_org_df['economic_sector_EN'].isna().sum()}\n")
print(ecg_org_df[ecg_org_df['economic_sector_EN'].isna()][['id', 'organization', 'economic_sector', 'economic_sector_EN']])

ecg_org_df.loc[ecg_org_df['id'] == 25050, 'economic_sector_EN'] = 'EDP / IT'
print(ecg_org_df[ecg_org_df['economic_sector_EN'].isna()][['id', 'organization', 'economic_sector', 'economic_sector_EN']])

# Check DE
print('\nGerman:\n')
print(f"Sum of missing and unassigned values: {ecg_org_df['economic_sector_DE'].isna().sum()}\n")
print(ecg_org_df[ecg_org_df['economic_sector_DE'].isna()][['id', 'organization', 'economic_sector', 'economic_sector_EN']])

ecg_org_df.loc[ecg_org_df['id'] == 25050, 'economic_sector_DE'] = 'EDV / IT'
print(ecg_org_df[ecg_org_df['economic_sector_DE'].isna()][['id', 'organization', 'economic_sector', 'economic_sector_EN']])

English:

Sum of missing and unassigned values: 11

          id                                    organization  \
22    164090                                      AliExpress   
48    164050                                           Apple   
49    164351                                           Apple   
310    28513                DROSG-PLÖCKINGER & PLÖCKINGER OG   
372    14805                             Fahnen-Gärtner GmbH   
383   163006                                             FBI   
391    14728  Fischer´s EM-Chiemgau – Christoph Fischer GmbH   
463   165419                                          Google   
727    28515                                Manfred Kofranek   
908   151681                                 pusch & Partner   
1194   25050                           visuellverstehen GmbH   

                             economic_sector economic_sector_EN  
22                                Will Smith                NaN  
48                            Jack Nicholson   

In [21]:
# balance_details
'''
To extract information from balance_details: Using pattern
CAN BE AUTOMATED (dependent on max number of balances: for balance1....n...)

# With dictionaries: https://stackoverflow.com/questions/6181935/how-do-you-create-different-variable-names-while-in-a-loop
d = {}
for x in range(1, 10):
    d["string{0}".format(x)] = "Hello"
'''

# Turn missing information into missing values
ecg_org_df.loc[ecg_org_df['balance_details'].astype(str).str.contains('liegen keine entsprechenden'), 'balance_details'] = None

# Check
ecg_org_df['balance_details']

# Define lists
number_of_balances_org_table = []

balance1_valid_until_date = []
balance2_valid_until_date = []
balance3_valid_until_date = []
balance4_valid_until_date = []
balance5_valid_until_date = []
balance6_valid_until_date = []
balance7_valid_until_date = []
balance8_valid_until_date = []
balance9_valid_until_date = []
balance10_valid_until_date = []

balance1_score = []
balance2_score = []
balance3_score = []
balance4_score = []
balance5_score = []
balance6_score = []
balance7_score = []
balance8_score = []
balance9_score = []
balance10_score = []

# Fill lists
for balance_detail in ecg_org_df['balance_details']:
    
    # number_of_balances_org_table
    try:
        if len(balance_detail) <=6:
            number_of_balances_org_table.append(1)
        elif len(balance_detail) <=12:
            number_of_balances_org_table.append(2)
        elif len(balance_detail) <=18:
            number_of_balances_org_table.append(3)
        elif len(balance_detail) <=24:
            number_of_balances_org_table.append(4)
        elif len(balance_detail) <=30:
            number_of_balances_org_table.append(5)
        elif len(balance_detail) <=36:
            number_of_balances_org_table.append(6)
        elif len(balance_detail) <=42:
            number_of_balances_org_table.append(7)
        elif len(balance_detail) <=48:
            number_of_balances_org_table.append(8)
        elif len(balance_detail) <=54:
            number_of_balances_org_table.append(9)
        elif len(balance_detail) <=60:
            number_of_balances_org_table.append(10)
        else:
            number_of_balances_org_table.append('>10')
    except:
        number_of_balances_org_table.append(0)
    
    # valid_until_date
    try:
        balance1_valid_until_date.append((balance_detail[4][1:11]).replace('.', '-'))
    except:
        balance1_valid_until_date.append(None)
    try:
        balance2_valid_until_date.append((balance_detail[10][1:11]).replace('.', '-'))
    except:
        balance2_valid_until_date.append(None)    
    try:
        balance3_valid_until_date.append((balance_detail[16][1:11]).replace('.', '-'))
    except:
        balance3_valid_until_date.append(None)        
    try:
        balance4_valid_until_date.append((balance_detail[22][1:11]).replace('.', '-'))
    except:
        balance4_valid_until_date.append(None)
    try:
        balance5_valid_until_date.append((balance_detail[28][1:11]).replace('.', '-'))
    except:
        balance5_valid_until_date.append(None)
    try:
        balance6_valid_until_date.append((balance_detail[34][1:11]).replace('.', '-'))
    except:
        balance6_valid_until_date.append(None)
    try:
        balance7_valid_until_date.append((balance_detail[40][1:11]).replace('.', '-'))
    except:
        balance7_valid_until_date.append(None)
    try:
        balance8_valid_until_date.append((balance_detail[46][1:11]).replace('.', '-'))
    except:
        balance8_valid_until_date.append(None)
    try:
        balance9_valid_until_date.append((balance_detail[52][1:11]).replace('.', '-'))
    except:
        balance9_valid_until_date.append(None)
    try:
        balance10_valid_until_date.append((balance_detail[58][1:11]).replace('.', '-'))
    except:
        balance10_valid_until_date.append(None)
    
    # score
    try:
        balance1_score.append(int(balance_detail[4][12:]))
    except:
        balance1_score.append(None)    
    try:
        balance2_score.append(int(balance_detail[10][12:]))
    except:
        balance2_score.append(None)
    try:
        balance3_score.append(int(balance_detail[16][12:]))
    except:
        balance3_score.append(None)    
    try:
        balance4_score.append(int(balance_detail[22][12:]))
    except:
        balance4_score.append(None)    
    try:
        balance5_score.append(int(balance_detail[28][12:]))
    except:
        balance5_score.append(None)    
    try:
        balance6_score.append(int(balance_detail[34][12:]))
    except:
        balance6_score.append(None)    
    try:
        balance7_score.append(int(balance_detail[40][12:]))
    except:
        balance7_score.append(None)
    try:
        balance8_score.append(int(balance_detail[46][12:]))
    except:
        balance8_score.append(None)
    try:
        balance9_score.append(int(balance_detail[52][12:]))
    except:
        balance9_score.append(None)
    try:
        balance10_score.append(int(balance_detail[58][12:]))
    except:
        balance10_score.append(None)

In [22]:
valid_number_of_balances = []
for entry in number_of_balances_org_table:
    if entry is not None and isinstance(entry, int):
        valid_number_of_balances.append(entry)

max_number_of_balances = max(valid_number_of_balances)

print(f'Maximum number of balances: {max_number_of_balances}')

print(pd.Series(valid_number_of_balances).value_counts().to_string())

print(f'NULL   {len(number_of_balances_org_table)-len(valid_number_of_balances)}')

Maximum number of balances: 8
1    975
2    158
0     75
3     40
4     13
5      2
6      2
7      1
8      1
NULL   0


In [23]:
# valid_until_date

# Check correct date format with an example case
print('Example date format of valid_until_date:')
print(balance1_valid_until_date[859])

Example date format of valid_until_date:
31-01-2025


In [24]:
# Correct scores being placed in valid_until_date lists due to missing values in valid_until_date 
i = 0
for element in balance1_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance1_valid_until_date[i] = None
            balance1_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance2_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance2_valid_until_date[i] = None
            balance2_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance3_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance3_valid_until_date[i] = None
            balance3_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance4_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance4_valid_until_date[i] = None
            balance4_score[i] = element
    except:
        pass
    i += 1
    
i = 0
for element in balance5_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance5_valid_until_date[i] = None
            balance5_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance6_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance6_valid_until_date[i] = None
            balance6_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance7_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance7_valid_until_date[i] = None
            balance7_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance8_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance8_valid_until_date[i] = None
            balance8_score[i] = element
    except:
        pass
    i += 1
    
i = 0
for element in balance9_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance9_valid_until_date[i] = None
            balance9_score[i] = element
    except:
        pass
    i += 1

i = 0
for element in balance10_valid_until_date:
    try:
        if len(element.strip()) <= 4:
            balance10_valid_until_date[i] = None
            balance10_score[i] = element
    except:
        pass
    i += 1

In [25]:
ecg_org_df['number_of_balances_org_table'] = number_of_balances_org_table

list_of_score_dictionaries = []

i=0
for element in range(len(ecg_org_df)):
    element_score_dictionary = {
        balance10_valid_until_date[i]: balance10_score[i], # Reverse order to avoid replacement in case of key=None, value= (a valid score)
        balance9_valid_until_date[i]: balance9_score[i],
        balance8_valid_until_date[i]: balance8_score[i],
        balance7_valid_until_date[i]: balance7_score[i],
        balance6_valid_until_date[i]: balance6_score[i],
        balance5_valid_until_date[i]: balance5_score[i],
        balance4_valid_until_date[i]: balance4_score[i],
        balance3_valid_until_date[i]: balance3_score[i],
        balance2_valid_until_date[i]: balance2_score[i],
        balance1_valid_until_date[i]: balance1_score[i]
        }
    
    # Removing values with missing value
    element_score_dictionary = {key: value for key, value in element_score_dictionary.items() if value is not None and value != ''}

    # Removing keys with missing value
    if None in element_score_dictionary:
        del element_score_dictionary[None]
    
    list_of_score_dictionaries.append(element_score_dictionary) 
    i += 1

list_of_score_dictionaries = [None if bool(x) is False else x for x in list_of_score_dictionaries]
    
ecg_org_df['balance_scores'] = list_of_score_dictionaries
list_of_score_dictionaries

[{'30-03-2020': 690},
 None,
 None,
 {'31-12-2014': 427},
 None,
 {'31-05-2017': 367, '30-11-2020': 476},
 {'31-10-2023': 358},
 {'30-10-2019': 491},
 None,
 {'30-06-2018': 556},
 {'30-09-2022': 392, '31-12-2025': 475},
 None,
 None,
 None,
 None,
 {'31-10-2025': 270},
 None,
 None,
 {'31-12-2015': 622},
 {'30-11-2023': 324},
 {'31-03-2024': 569},
 {'30-04-2023': 377},
 None,
 {'01-04-2019': 446},
 {'31-07-2024': 266},
 {'31-07-2023': 291},
 {'31-07-2025': 242},
 {'30-12-2018': 627, '31-05-2021': 444},
 {'20-01-2016': 548},
 {'31-12-2020': 304},
 {'30-11-2024': 444},
 {'30-11-2025': 338},
 None,
 None,
 {'31-01-2018': 405, '31-07-2020': 489, '31-05-2023': 437},
 {'31-12-2024': 229},
 {'30-03-2021': 702},
 {'31-12-2021': 290},
 {'30-04-2023': 320},
 {'31-07-2019': 415},
 {'24-05-2018': 840},
 {'20-10-2020': 348},
 {'31-12-2015': 480, '30-11-2018': 568},
 {'31-07-2024': 509},
 {'31-07-2025': 305},
 {'31-07-2024': 257},
 None,
 {'30-10-2020': 577},
 None,
 None,
 {'31-07-2020': 728},
 {'3

#### Creating a link-id dictionary (important for scraping the balances table)

In [26]:
# Extracting keys from links
organization_keys = []

for link in organization_links:
    organization_keys.append(link.split('=')[2])
    
# Creating dictionary
organization_key_id_dict = dict(zip(organization_keys, list(ecg_org_df['id'])))
organization_key_id_dict

{'9bdu6': 14885,
 '6olka': 14886,
 'w5zyz': 35275,
 'rasmd': 44668,
 'da7lg': 29652,
 '3sz2i': 14887,
 'vjsi5': 46986,
 'o2z0h': 92361,
 'bvreg': 14888,
 '5lef1': 40981,
 'y6ccf': 14973,
 '45pi7': 92389,
 'caqa6': 150227,
 'fwxft': 30337,
 '2y42g': 29635,
 'pi1sy': 132863,
 'b9wov': 92421,
 'l4ni9': 92373,
 'agvoq': 36548,
 'oosvv': 37075,
 'z18uw': 58579,
 't6kww': 29850,
 'flhbb': 164090,
 'm5llk': 92467,
 'kmpbs': 78313,
 'o2pj4': 35674,
 'ta3z6': 113132,
 '7uwre': 14665,
 'mllle': 92491,
 'c8r03': 14889,
 'wofj2': 93957,
 'rgfx9': 139397,
 '7gbj6': 92475,
 '9c0ia': 92443,
 '4pvzw': 14818,
 '709cf': 91614,
 'n11c4': 14819,
 '9sm8d': 14772,
 'hggub': 29214,
 'bex4a': 30925,
 'e7ozf': 92507,
 'mtwcv': 92457,
 'n04xj': 37702,
 'npt5o': 79370,
 '3c0ai': 126462,
 'pbgd8': 82864,
 'vn77v': 92455,
 '3qi4h': 14820,
 'x7l86': 164050,
 'e1bo8': 164351,
 'uo8hd': 14821,
 '5d0zm': 14884,
 'etuco': 14822,
 'lgdte': 92371,
 'rzuqj': 166562,
 'brwy0': 14737,
 'u5bvz': 44670,
 'xcnhm': 37707,
 '87t

#### Remove irrelevant entries

In [27]:
# Delete rows with "YYYY" entries in organization (used by ecg admins for testing purposes)
test_entries = ecg_org_df[ecg_org_df['organization'].str.contains('YYY')]
if len(test_entries) >= 1:   
    print(f'{len(test_entries)} test entries detected. Deleted the following entries:')
    display(test_entries)
    
    ecg_org_df = ecg_org_df.drop(test_entries.index).reset_index(drop=True)
    
else:
    print('No test entries detected.')
    
# Delete rows without any balance in the system
no_balance_entries = ecg_org_df[ecg_org_df['balance_details'].isna()]
if len(no_balance_entries) >= 1:
    print(f'\n{len(no_balance_entries)} entries without any balance detected. Deleted the following entries:')
    display(no_balance_entries)
    
    ecg_org_df = ecg_org_df.drop(no_balance_entries.index).reset_index(drop=True)

else:
    print('\nNo entries without any balance in the system detected.')

2 test entries detected. Deleted the following entries:


,id,organization,organization_link,country_code,postal_code,city,company_size,economic_sector,balance_details,country_name,company_size_category,employees,economic_sector_EN,economic_sector_DE,number_of_balances_org_table,balance_scores
1253,159682,YYYY Test,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,1111,YYYY Testort,(4)26-50,Beratung / Consulting,"[, , …159681, M5.0 VollbilanzBesuchsaudit]",Germany,Small (≤50),26-50,Consulting,Beratung / Consulting,1,None
1254,159686,YYYY Test,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,1111,YYYY Testort,(4)26-50,Beratung / Consulting,"[, , …159685, M5.0 KompaktbilanzBesuchsaudit, ...",Germany,Small (≤50),26-50,Consulting,Beratung / Consulting,3,None



75 entries without any balance detected. Deleted the following entries:


,id,organization,organization_link,country_code,postal_code,city,company_size,economic_sector,balance_details,country_name,company_size_category,employees,economic_sector_EN,economic_sector_DE,number_of_balances_org_table,balance_scores
11,92389,AFEF-EBC,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,37187,Miranda de Azán,(8) Todas las tallas,Otros,None,Spain,None,None,Other,Sonstige,0,None
16,92421,Agresta Soc. Cooperativa,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,38004,Santa Cruz de Tenerife,(8) Todas las tallas,Otros,None,Spain,None,None,Other,Sonstige,0,None
17,92373,Agricología José Manuel Bisetto,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,03759,Benidoleig,(8) Todas las tallas,Otros,None,Spain,None,None,Other,Sonstige,0,None
22,164090,AliExpress,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,None,Minsk,AliExpress,Will Smith,None,Spain,NaN,NaN,NaN,NaN,0,None
32,92475,Alter Consulta,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,28014,Madrid,(1)1-2,Otros,None,Spain,Micro (≤10),1-2,Other,Sonstige,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174,92383,Urbaser S.A,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,38110,Santa Cruz de Tenerife,(8) Todas las tallas,Otros,None,Spain,None,None,Other,Sonstige,0,None
1177,92463,USS,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,35010,Las Palmas de Gran Canaria,(1)1-2,Otros,None,Spain,Micro (≤10),1-2,Other,Sonstige,0,None
1184,92461,"Veritable, SL",https://audit.ecogood.org/firmenauskunft-2/?la...,ES,08028,Barcelona,(1)1-2,Otros,None,Spain,Micro (≤10),1-2,Other,Sonstige,0,None
1191,92375,Viña ecológica Miguel Poveda Justamante,https://audit.ecogood.org/firmenauskunft-2/?la...,ES,03640,Monòver,(8) Todas las tallas,Otros,None,Spain,None,None,Other,Sonstige,0,None


#### Final cleaning steps

In [28]:
# Choose relevant columns
ecg_org_df = ecg_org_df[[
    'id',
    'organization',
    'organization_link',
    'country_code',
    'country_name',
    'postal_code',
    'city',
    'company_size_category',
    'employees',
    'economic_sector_EN',
    'economic_sector_DE',
    'number_of_balances_org_table',
    'balance_scores'
    ]]

In [29]:
# Checks after cleaning and transformation
print(f'Datatype: \n{ecg_org_df.dtypes}\n')

Datatype: 
id                               int64
organization                    object
organization_link               object
country_code                    object
country_name                    object
postal_code                     object
city                            object
company_size_category           object
employees                       object
economic_sector_EN              object
economic_sector_DE              object
number_of_balances_org_table     int64
balance_scores                  object
dtype: object



In [30]:
print(f'Missing values: \n{ecg_org_df.isna().sum()}\n')

Missing values: 
id                                0
organization                      0
organization_link                 0
country_code                      0
country_name                      0
postal_code                       0
city                              0
company_size_category            27
employees                        27
economic_sector_EN                2
economic_sector_DE                2
number_of_balances_org_table      0
balance_scores                  130
dtype: int64



In [31]:
# Replace all empty strings with missing values

# Check for empty strings
if ecg_org_df.eq('').any(axis=None) == False:
    print('No empty strings ("") in dataframe.')

else:
    pd.set_option('display.max_columns', None) # Showing all columns
    print(ecg_org_df[ecg_org_df.eq('').any(axis=1)])
    pd.reset_option('max_columns')

    ecg_org_df = ecg_org_df.replace('', np.nan)

    pd.set_option('display.max_columns', None) # Showing all columns
    print(ecg_org_df[ecg_org_df.eq('').any(axis=1)])
    pd.reset_option('max_columns')

No empty strings ("") in dataframe.


In [32]:
# Turn None type values into NaN
ecg_org_df = ecg_org_df.fillna(value=np.nan)

In [33]:
# Final check
ecg_org_df

,id,organization,organization_link,country_code,country_name,postal_code,city,company_size_category,employees,economic_sector_EN,economic_sector_DE,number_of_balances_org_table,balance_scores
0,14885,4e solutions GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,70794,Filderstadt,Micro (≤10),3-10,Trade / Consumption,Handel / Konsum,1,{'30-03-2020': 690}
1,14886,4plus5,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,89077,Ulm,Micro (≤10),3-10,Construction / Architecture,Baugewerbe / Architektur,1,NaN
2,35275,A & K Engemann GbR,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,34439,Willebadessen,Micro (≤10),3-10,Nutrition / Agriculture / Forestry,Nahrungsmittel / Land / Forstwirtschaft,1,NaN
3,44668,A&P Steuerberatungsgesellschaft mbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,14480,Potsdam,Small (≤50),11-25,Consulting,Beratung / Consulting,1,{'31-12-2014': 427}
4,29652,AAP-ARCHITEKTEN ZT-GMBH,https://audit.ecogood.org/firmenauskunft-2/?la...,AT,Austria,1080,Wien,Small (≤50),11-25,Construction / Architecture,Baugewerbe / Architektur,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1185,92844,Zimmerei Diedrich – Die Gesundhausbauer GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,37434,Rüdershausen,Small (≤50),11-25,Construction / Architecture,Baugewerbe / Architektur,1,{'31-10-2024': 282}
1186,45156,zimmerwerkstatt Nicola Bannier und Michael Weber,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,29456,Hitzacker,Micro (≤10),1-2,Craft,Handwerk,1,{'28-02-2016': 534}
1187,165659,ZORA Kinder- und Jugendhilfe gGmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,17489,Greifswald,Mid (≤250),51-100,Health / Social Affairs / Nursing,Gesundheitswesen / Soziales / Pflege,1,NaN
1188,14718,Zukunftswerk eG,https://audit.ecogood.org/firmenauskunft-2/?la...,DE,Germany,82319,Starnberg,Micro (≤10),3-10,Consulting,Beratung / Consulting,1,{'30-04-2021': 432}


#### Export

In [34]:
ecg_org_df.to_csv('ecg_org_df.csv', index=False)

## Scraping balance-oriented view
https://audit.ecogood.org/firmenauskunft/

#### URL creation

In [35]:
# URLs (first page) & base URLs / indicators (when browsing through the table)

# audit
audit_url = 'https://audit.ecogood.org/firmenauskunft/?ser_fvz_pagesize=50#1574317575063-06b4d9e5-e677'
audit_base_url = 'https://audit.ecogood.org/firmenauskunft/?ser_fvz_pagesize=50&frm-page-4770='
audit_indicator = '#1574317575063-06b4d9e5-e677' # attach directly after page no.

# peer
peer_url = 'https://audit.ecogood.org/firmenauskunft/?ser_fvz_pagesize=50#1574317575155-9e81c480-beb1'
peer_base_url = 'https://audit.ecogood.org/firmenauskunft/?frm-page-4772='
peer_indicator = '#1574317575155-9e81c480-beb1' # attach directly after page no.

In [36]:
# Create list with audit URLs to scrape through (as max. 50 entries per page are displayed)

# Creating soup
audit_response = requests.get(audit_url)
audit_soup = BeautifulSoup(audit_response.content, "html.parser")
print(f'Response status (audit): {audit_response.status_code}')
print()

# Finding out number of total entries
total_audit_entries = int(list(audit_soup.select('div[style="overflow-x:auto;"] table[id="Liste_Firmen_Audittool"]')[0])[0][-4:])
print(f'Total entries in audit database: {total_audit_entries}')

# Finding out number of URLs (pages)
number_of_audit_urls = int(total_audit_entries/50)+1
print(f'Number of audit table pages: {number_of_audit_urls}')
print()

dynamic_audit_url_list = [audit_base_url + str(i) + audit_indicator for i in range(1,number_of_audit_urls+1)]
print(f'Audit URL list created. Length of dynamic_audit_url_list: {len(dynamic_audit_url_list)}')

Response status (audit): 200

Total entries in audit database: 688
Number of audit table pages: 14

Audit URL list created. Length of dynamic_audit_url_list: 14


In [37]:
# Create list with peer URLs to scrape through (as max. 50 entries per page are displayed)

# Creating soup
peer_response = requests.get(peer_url)
peer_soup = BeautifulSoup(peer_response.content, "html.parser")
print(f'Response status (peer): {peer_response.status_code}')
print()

# Finding out number of total entries
total_peer_entries = int(list(peer_soup.select('div[style="overflow-x:auto;"] table[id="Liste_Firmen_Audittool"]')[1])[0][-4:])
print(f'Total entries in peer database: {total_peer_entries}')

# Finding out number of URLs (pages)
number_of_peer_urls = int(total_peer_entries/50)+1
print(f'Number of peer table pages: {number_of_peer_urls}')
print()

dynamic_peer_url_list = [peer_base_url + str(i) + peer_indicator for i in range(1,number_of_peer_urls+1)]
print(f'Peer URL list created. Length of dynamic_peer_url_list: {len(dynamic_peer_url_list)}')

Response status (peer): 200

Total entries in peer database: 769
Number of peer table pages: 16

Peer URL list created. Length of dynamic_peer_url_list: 16


#### Checks

In [38]:
# Checking tables
for table in audit_soup.select('table')[0:2]: # Table 0: audit, table 1: peer 
    print(table.get('id'))

Liste_Firmen_Audittool
Liste_Firmen_Audittool


In [39]:
# Checking column titles
audit_data = audit_soup.select('table')[0]
column_titles = []
for header in audit_data.select('tr')[0]:
    if header.get_text().strip() != '':
        column_titles.append(header.get_text().strip())

#print(len(titles))
print(column_titles)

['Bilanz', 'MA', 'Branche', 'Land-PLZ Ort', 'Organisation', 'Einreichung', 'Gültigkeit', 'Dokumente']


#### Scraping the first page (for testing)

In [40]:
# Selecting table with relevant information and assigning variable
balance_data = audit_soup.select('table')[0]

# Creating empty lists
balance_details = []
organizations = []
submission_date = []
valid_until_date = []
documents = []

# Scraping main data
for row in balance_data.select('tbody tr'):
    balance_details.append(row.get_text().strip().split('\n')[0])
    organizations.append(row.get_text().strip().split('\n')[4])
    submission_date.append(row.get_text().strip().split('\n')[5])
    valid_until_date.append(row.get_text().strip().split('\n')[6])
    documents.append(row.get_text().strip().split('\n')[7])
        
# Check
#documents

In [41]:
'''

# For ids
balance_data = audit_soup.select('table')[0]
organization_links = []

for a in balance_data.find_all('a'):
    if 'firmenauskunft' in a.get('href'):
        organization_links.append(a.get('href'))

len(organization_links)

'''

"\n\n# For ids\nbalance_data = audit_soup.select('table')[0]\norganization_links = []\n\nfor a in balance_data.find_all('a'):\n    if 'firmenauskunft' in a.get('href'):\n        organization_links.append(a.get('href'))\n\nlen(organization_links)\n\n"

#### Scraping

In [42]:
# Creating empty lists and dictionary
balance_audit_types = []
balance_details = []
organizations = []
organization_links = []
submission_dates = []
valid_until_dates = []
documents = []

In [43]:
# Main scraping

# audit
i=0
for url in dynamic_audit_url_list:
    response = requests.get(url)
    audit_soup = BeautifulSoup(response.content, "html.parser")
    audit_data = audit_soup.select('table')[0]
    
    for row in audit_data.select('tbody tr'):
        balance_audit_types.append('Audit')
        balance_details.append(row.get_text().strip().split('\n')[0])
        organizations.append(row.get_text().strip().split('\n')[4])
        submission_dates.append(row.get_text().strip().split('\n')[5])
        valid_until_dates.append(row.get_text().strip().split('\n')[6])
        documents.append(row.get_text().strip().split('\n')[7])
        
    for a in audit_data.find_all('a'):
        if 'firmenauskunft' in a.get('href'):
            organization_links.append(a.get('href'))
                
    # Show status
    i += 1
    print(f'Page {i}/{len(dynamic_audit_url_list)+len(dynamic_peer_url_list)} scraped.')

# peer
for url in dynamic_peer_url_list:
    response = requests.get(url)
    peer_soup = BeautifulSoup(response.content, "html.parser")
    peer_data = peer_soup.select('table')[1]
    
    for row in peer_data.select('tbody tr'):
        balance_audit_types.append('Peer')
        balance_details.append(row.get_text().strip().split('\n')[0])
        organizations.append(row.get_text().strip().split('\n')[4])
        submission_dates.append(row.get_text().strip().split('\n')[5])
        valid_until_dates.append(row.get_text().strip().split('\n')[6])
        documents.append(row.get_text().strip().split('\n')[7])
                
    for a in peer_data.find_all('a'):
        if 'firmenauskunft' in a.get('href'):
            organization_links.append(a.get('href'))
        
    # Show status
    i += 1
    print(f'Page {i}/{len(dynamic_audit_url_list)+len(dynamic_peer_url_list)} scraped.')        
        
# Check
print(f'\nAll pages scraped. Length: {len(organizations)} entries.')

Page 1/30 scraped.
Page 2/30 scraped.
Page 3/30 scraped.
Page 4/30 scraped.
Page 5/30 scraped.
Page 6/30 scraped.
Page 7/30 scraped.
Page 8/30 scraped.
Page 9/30 scraped.
Page 10/30 scraped.
Page 11/30 scraped.
Page 12/30 scraped.
Page 13/30 scraped.
Page 14/30 scraped.
Page 15/30 scraped.
Page 16/30 scraped.
Page 17/30 scraped.
Page 18/30 scraped.
Page 19/30 scraped.
Page 20/30 scraped.
Page 21/30 scraped.
Page 22/30 scraped.
Page 23/30 scraped.
Page 24/30 scraped.
Page 25/30 scraped.
Page 26/30 scraped.
Page 27/30 scraped.
Page 28/30 scraped.
Page 29/30 scraped.
Page 30/30 scraped.

All pages scraped. Length: 1457 entries.


#### (Not necessary anymore)

In [44]:
'''

# Test scrape for one id
url = organization_links[0]

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

org_id = int(soup.select('table[id="RW_Firmenliste"] tbody tr td')[0].get_text())
org_id

'''

'\n\n# Test scrape for one id\nurl = organization_links[0]\n\nresponse = requests.get(url)\nsoup = BeautifulSoup(response.content, "html.parser")\n\norg_id = int(soup.select(\'table[id="RW_Firmenliste"] tbody tr td\')[0].get_text())\norg_id\n\n'

In [45]:
'''

# Scraping for ids
organization_ids = []

i=0
for link in organization_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    
    try:
        organization_ids.append(int(soup.select('table[id="RW_Firmenliste"] tbody tr td')[0].get_text()))
    
    except:
        organization_ids.append(None)
    
    i += 1
    if i%100 == 0: 
        print(f'Scraped {i} links')
        
print('All links scraped.')

'''

'\n\n# Scraping for ids\norganization_ids = []\n\ni=0\nfor link in organization_links:\n    response = requests.get(link)\n    soup = BeautifulSoup(response.content, "html.parser")\n    \n    try:\n        organization_ids.append(int(soup.select(\'table[id="RW_Firmenliste"] tbody tr td\')[0].get_text()))\n    \n    except:\n        organization_ids.append(None)\n    \n    i += 1\n    if i%100 == 0: \n        print(f\'Scraped {i} links\')\n        \nprint(\'All links scraped.\')\n\n'

In [46]:
# Creating basic dataframe
ecg_bal_df = pd.DataFrame({#'id': ids, 
                'balance_audit_type': balance_audit_types,
                'balance_details': balance_details,
                'organization': organizations,
                'organization_link': organization_links,
                'submission_date': submission_dates,
                'valid_until_date': valid_until_dates,
                'documents': documents
                })
print(f'Shape: {ecg_bal_df.shape}\n')
print(f'Datatype: \n{ecg_bal_df.dtypes}\n')
display(ecg_bal_df)

Shape: (1457, 7)

Datatype: 
balance_audit_type    object
balance_details       object
organization          object
organization_link     object
submission_date       object
valid_until_date      object
documents             object
dtype: object



,balance_audit_type,balance_details,organization,organization_link,submission_date,valid_until_date,documents
0,Audit,M5.0 Kompaktbilanz,ALLMENDA Social Business eG,https://audit.ecogood.org/firmenauskunft-2/?la...,2018.10.28,2021.05.31,Bericht / Testat
1,Audit,M5.0 Vollbilanz,Backkultur,https://audit.ecogood.org/firmenauskunft-2/?la...,2019.01.31,2021.05.31,Bericht / Testat
2,Audit,M5.0 Vollbilanz,Bernhard Oberrauch,https://audit.ecogood.org/firmenauskunft-2/?la...,2018.09.01,2023.03.31,Bericht / Testat
3,Audit,M5.0 Vollbilanz,buch7.de GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,2018.11.16,2020.12.31,Bericht / Testat
4,Audit,M5.0 Kompaktbilanz,"Cosyma, EPU",https://audit.ecogood.org/firmenauskunft-2/?la...,2018.07.01,2020.10.31,Bericht / Testat
...,...,...,...,...,...,...,...
1452,Peer,M5.0 Vollbilanz,dng die neue gesellschaft,https://audit.ecogood.org/firmenauskunft-2/?la...,2024.02.27,,/
1453,Peer,M5.0 Vollbilanz,stattbau münchen GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,2024.02.27,,/
1454,Peer,M5.0 Vollbilanz,terratax Steuerberatungsgesellschaft mbH,https://audit.ecogood.org/firmenauskunft-2/?la...,2024.02.27,,/
1455,Peer,M5.0 Kompaktbilanz,Daniel Börnert mindfulbranding,https://audit.ecogood.org/firmenauskunft-2/?la...,2024.03.11,,/


#### Data cleaning and initial transformation

In [47]:
# balance_type
balance_types = [detail.split(' ')[0].strip() for detail in balance_details]
ecg_bal_df['balance_type'] = balance_types

balance_type_dict = {
    'M5.0': 'M5.0',
    '4.1': 'M4.1',
    '4.0': 'M4.0',
    '3.0': 'M3.0',
    'Gemeinde': 'M1.2'
    }

ecg_bal_df['balance_type'] = ecg_bal_df['balance_type'].apply(lambda x: balance_type_dict.get(x,x))

# Check
ecg_bal_df['balance_type'].value_counts()
#ecg_bal_df['balance_type'].isna().sum()

M5.0    1085
M4.1     287
M4.0      71
M3.0      10
M1.2       4
Name: balance_type, dtype: int64

In [48]:
# balance_version
balance_versions = [detail.split(' ')[1].strip() for detail in balance_details]
ecg_bal_df['balance_version'] = balance_versions

balance_version_dict = {
    'Kompaktbilanz': 'Compact',
    'Standard': 'Standard',
    'Vollbilanz': 'Full',
    'Balance': 'Full', # It seems to be "balance completo" in organization-oriented table
    'Compact': 'Compact',
    'M1.2': 'Municipal',
    'Full': 'Full'
    }

ecg_bal_df['balance_version'] = ecg_bal_df['balance_version'].apply(lambda x: balance_version_dict.get(x,x))

# Check
ecg_bal_df['balance_version'].value_counts()
#ecg_bal_df['balance_version'].isna().sum()

Compact      627
Full         458
Standard     368
Municipal      4
Name: balance_version, dtype: int64

In [49]:
# organization_id

# Extracting keys from links
organization_keys = []

for link in organization_links:
    organization_keys.append(link.split('=')[2])

# Mapping keys-ids dictionary from organization-oriented view
ecg_bal_df['organization_id'] = organization_keys
#ecg_bal_df['organization_id'] = ecg_bal_df['organization_id'].apply(lambda x: organization_key_id_dict.get(x,x))
ecg_bal_df['organization_id'] = ecg_bal_df['organization_id'].map(organization_key_id_dict)

#Check
#ecg_bal_df['organization_id'].tail(60)
ecg_bal_df['organization_id'].isna().sum()

65

In [50]:
# submission_date
submission_dates_reordered = [f'{date.split(".")[2].strip()}-{date.split(".")[1].strip()}-{date.split(".")[0].strip()}' if date!='' else None for date in submission_dates]
ecg_bal_df['submission_date'] = submission_dates_reordered
ecg_bal_df['submission_date']

0       28-10-2018
1       31-01-2019
2       01-09-2018
3       16-11-2018
4       01-07-2018
           ...    
1452    27-02-2024
1453    27-02-2024
1454    27-02-2024
1455    11-03-2024
1456    11-03-2024
Name: submission_date, Length: 1457, dtype: object

In [51]:
# valid_until_date
valid_until_dates_reordered = [f'{date.split(".")[2].strip()}-{date.split(".")[1].strip()}-{date.split(".")[0].strip()}' if date!='' else None for date in valid_until_dates]
ecg_bal_df['valid_until_date'] = valid_until_dates_reordered
ecg_bal_df['valid_until_date']

0       31-05-2021
1       31-05-2021
2       31-03-2023
3       31-12-2020
4       31-10-2020
           ...    
1452          None
1453          None
1454          None
1455          None
1456          None
Name: valid_until_date, Length: 1457, dtype: object

In [52]:
# issue_date
issue_dates = [f'{date.split("-")[0]}-{date.split("-")[1]}-{str(int(date.split("-")[2])-2)}' if date is not None else None for date in valid_until_dates_reordered]
ecg_bal_df['issue_date'] = issue_dates
ecg_bal_df['issue_date']

0       31-05-2019
1       31-05-2019
2       31-03-2021
3       31-12-2018
4       31-10-2018
           ...    
1452          None
1453          None
1454          None
1455          None
1456          None
Name: issue_date, Length: 1457, dtype: object

In [53]:
# documents
def documents_replace(x):
    x = x.replace('Bericht', 'Report')
    x = x.replace('Zertifikat', 'Certificate')
    x = x.replace('Testat', 'Attestation')
    x = x.replace('/','+')
    x = x.replace(' ', '')
    return x

documents_reformatted = [documents_replace(info).strip() for info in documents]
documents_reformatted = [info.replace('+', '').strip() if info.endswith('+') else info for info in documents_reformatted]
documents_reformatted = [info.replace('+', '').strip() if info.startswith('+') else info for info in documents_reformatted]
documents_reformatted = [None if info=='' else info for info in documents_reformatted]
ecg_bal_df['documents'] = documents_reformatted
ecg_bal_df['documents'].head()

0    Report+Attestation
1    Report+Attestation
2    Report+Attestation
3    Report+Attestation
4    Report+Attestation
Name: documents, dtype: object

In [54]:
# balance_score
id_balance_score_mapping = dict(zip(ecg_org_df['id'], ecg_org_df['balance_scores']))

ecg_bal_df['balance_score'] = ecg_bal_df['organization_id']
ecg_bal_df['balance_score'] = ecg_bal_df['balance_score'].apply(lambda x: id_balance_score_mapping.get(x,x))

for i in range(len(ecg_bal_df['balance_score'])):
    if isinstance(ecg_bal_df.loc[i, 'valid_until_date'],str) and isinstance(ecg_bal_df.loc[i, 'balance_score'],dict):
        if ecg_bal_df.loc[i, 'valid_until_date'] in ecg_bal_df.loc[i, 'balance_score']:
            ecg_bal_df.loc[i, 'balance_score'] = ecg_bal_df.loc[i, 'balance_score'][ecg_bal_df.loc[i, 'valid_until_date']]
        else:
            ecg_bal_df.loc[i, 'balance_score'] = np.nan
    else:
        ecg_bal_df.loc[i, 'balance_score'] = np.nan

# Check
ecg_bal_df['balance_score'].head(60)
ecg_bal_df['balance_score'].isna().sum()

196

#### Final cleaning steps

In [55]:
ecg_bal_df

,balance_audit_type,balance_details,organization,organization_link,submission_date,valid_until_date,documents,balance_type,balance_version,organization_id,issue_date,balance_score
0,Audit,M5.0 Kompaktbilanz,ALLMENDA Social Business eG,https://audit.ecogood.org/firmenauskunft-2/?la...,28-10-2018,31-05-2021,Report+Attestation,M5.0,Compact,14665.0,31-05-2019,444
1,Audit,M5.0 Vollbilanz,Backkultur,https://audit.ecogood.org/firmenauskunft-2/?la...,31-01-2019,31-05-2021,Report+Attestation,M5.0,Full,14666.0,31-05-2019,453
2,Audit,M5.0 Vollbilanz,Bernhard Oberrauch,https://audit.ecogood.org/firmenauskunft-2/?la...,01-09-2018,31-03-2023,Report+Attestation,M5.0,Full,14667.0,31-03-2021,520
3,Audit,M5.0 Vollbilanz,buch7.de GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,16-11-2018,31-12-2020,Report+Attestation,M5.0,Full,14668.0,31-12-2018,517
4,Audit,M5.0 Kompaktbilanz,"Cosyma, EPU",https://audit.ecogood.org/firmenauskunft-2/?la...,01-07-2018,31-10-2020,Report+Attestation,M5.0,Compact,14669.0,31-10-2018,567
...,...,...,...,...,...,...,...,...,...,...,...,...
1452,Peer,M5.0 Vollbilanz,dng die neue gesellschaft,https://audit.ecogood.org/firmenauskunft-2/?la...,27-02-2024,None,None,M5.0,Full,164463.0,None,NaN
1453,Peer,M5.0 Vollbilanz,stattbau münchen GmbH,https://audit.ecogood.org/firmenauskunft-2/?la...,27-02-2024,None,None,M5.0,Full,164465.0,None,NaN
1454,Peer,M5.0 Vollbilanz,terratax Steuerberatungsgesellschaft mbH,https://audit.ecogood.org/firmenauskunft-2/?la...,27-02-2024,None,None,M5.0,Full,164468.0,None,NaN
1455,Peer,M5.0 Kompaktbilanz,Daniel Börnert mindfulbranding,https://audit.ecogood.org/firmenauskunft-2/?la...,11-03-2024,None,None,M5.0,Compact,166560.0,None,NaN


In [56]:
# Choose relevant columns
ecg_bal_df = ecg_bal_df[[
    'organization_id',
    'organization',
    'balance_audit_type',
    'balance_type',
    'balance_version',
    'submission_date',
    'issue_date',
    'valid_until_date',
    'balance_score',
    'documents'
    ]]

In [57]:
# Checks after cleaning and transformation
print(f'Datatype: \n{ecg_bal_df.dtypes}\n')

Datatype: 
organization_id       float64
organization           object
balance_audit_type     object
balance_type           object
balance_version        object
submission_date        object
issue_date             object
valid_until_date       object
balance_score          object
documents              object
dtype: object



In [58]:
print(f'Missing values: \n{ecg_bal_df.isna().sum()}\n')

Missing values: 
organization_id        65
organization            0
balance_audit_type      0
balance_type            0
balance_version         0
submission_date         0
issue_date            169
valid_until_date      169
balance_score         196
documents             174
dtype: int64



In [59]:
# Replace all empty strings with missing values

# Check for empty strings
if ecg_bal_df.eq('').any(axis=None) == False:
    print('No empty strings ("") in dataframe.')

else:
    pd.set_option('display.max_columns', None) # Showing all columns
    print(ecg_bal_df[ecg_bal_df.eq('').any(axis=1)])
    pd.reset_option('max_columns')

    ecg_bal_df = ecg_bal_df.replace('', np.nan)

    pd.set_option('display.max_columns', None) # Showing all columns
    print(ecg_bal_df[ecg_bal_df.eq('').any(axis=1)])
    pd.reset_option('max_columns')

No empty strings ("") in dataframe.


In [60]:
# Turn None type values into NaN
ecg_bal_df = ecg_bal_df.fillna(value=np.nan)

In [61]:
# Final check
ecg_bal_df

,organization_id,organization,balance_audit_type,balance_type,balance_version,submission_date,issue_date,valid_until_date,balance_score,documents
0,14665.0,ALLMENDA Social Business eG,Audit,M5.0,Compact,28-10-2018,31-05-2019,31-05-2021,444.0,Report+Attestation
1,14666.0,Backkultur,Audit,M5.0,Full,31-01-2019,31-05-2019,31-05-2021,453.0,Report+Attestation
2,14667.0,Bernhard Oberrauch,Audit,M5.0,Full,01-09-2018,31-03-2021,31-03-2023,520.0,Report+Attestation
3,14668.0,buch7.de GmbH,Audit,M5.0,Full,16-11-2018,31-12-2018,31-12-2020,517.0,Report+Attestation
4,14669.0,"Cosyma, EPU",Audit,M5.0,Compact,01-07-2018,31-10-2018,31-10-2020,567.0,Report+Attestation
...,...,...,...,...,...,...,...,...,...,...
1452,164463.0,dng die neue gesellschaft,Peer,M5.0,Full,27-02-2024,NaN,NaN,NaN,NaN
1453,164465.0,stattbau münchen GmbH,Peer,M5.0,Full,27-02-2024,NaN,NaN,NaN,NaN
1454,164468.0,terratax Steuerberatungsgesellschaft mbH,Peer,M5.0,Full,27-02-2024,NaN,NaN,NaN,NaN
1455,166560.0,Daniel Börnert mindfulbranding,Peer,M5.0,Compact,11-03-2024,NaN,NaN,NaN,NaN


#### Export

In [62]:
ecg_bal_df.to_csv('ecg_bal_df.csv', index=False)

## Post-export checks

In [65]:
# Do the number of balances between both tables differ in any cases?
org_balance_count = ecg_org_df[['id', 'organization', 'number_of_balances_org_table', 'balance_scores']]
org_balance_count = org_balance_count.sort_values('id').reset_index(drop=True)
org_balance_count.columns = ['id', 'organization', 'balance_count_org', 'details_org']
org_balance_count['id'] = org_balance_count['id'].astype(float)
org_balance_count['details_org'] = org_balance_count['details_org'].apply(lambda x: ", ".join(list(x.keys())) if isinstance(x, dict) else x)

bal_balance_count = ecg_bal_df[['organization_id', 'valid_until_date']]
bal_balance_count['valid_until_date'] = (bal_balance_count['valid_until_date'] + ', ')
bal_balance_count = bal_balance_count.groupby('organization_id').agg(balance_count=('organization_id', 'count'), valid_info=('valid_until_date', 'sum'))
bal_balance_count['valid_info'] = bal_balance_count['valid_info'].apply(lambda x: x[:-2] if isinstance(x, str) else x)
bal_balance_count.reset_index(inplace=True)
bal_balance_count = bal_balance_count.sort_values('organization_id').reset_index(drop=True)
bal_balance_count.columns = ['id', 'balance_count_bal','details_bal']
bal_balance_count['balance_count_bal'].fillna(0, inplace=True)
bal_balance_count['balance_count_bal'] = bal_balance_count['balance_count_bal'].astype(int)

joined_balance_counts = pd.merge(org_balance_count, bal_balance_count, on='id', how='outer')
joined_balance_counts['balance_count_bal'] = joined_balance_counts['balance_count_bal'].fillna(0).astype(int)
joined_balance_counts = joined_balance_counts[['id', 'organization', 'balance_count_org', 'balance_count_bal', 'details_org', 'details_bal']]
joined_balance_counts['details_bal'] = joined_balance_counts['details_bal'].apply(lambda x: np.nan if isinstance(x, int) else x)
joined_balance_counts['id'] = joined_balance_counts['id'].astype(int)

joined_balance_counts.to_csv('checks/joined_balance_counts.csv', index=False)

print(f'Balances count in ecg_org_df: {joined_balance_counts["balance_count_org"].sum()}')
print(f'Balances count in ecg_bal_df: {joined_balance_counts["balance_count_bal"].sum()}')

diff_in_balance_counts = joined_balance_counts[joined_balance_counts['balance_count_org'] != joined_balance_counts["balance_count_bal"]]
print(f'\nFound differences in {len(diff_in_balance_counts)} cases between the dataframes:')
display(diff_in_balance_counts.head(50))
diff_in_balance_counts.to_csv('checks/diff_in_balance_counts.csv', index=False)

Balances count in ecg_org_df: 1496
Balances count in ecg_bal_df: 1392

Found differences in 102 cases between the dataframes:


/tmp/ipykernel_4821/2934918000.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bal_balance_count['valid_until_date'] = (bal_balance_count['valid_until_date'] + ', ')


,id,organization,balance_count_org,balance_count_bal,details_org,details_bal
0,14654,Genusshandwerker,2,1,31-03-2018,31-03-2018
8,14662,Landgut Stober Kontor GmbH & Co. Betriebs KG,2,1,30-04-2021,30-04-2021
14,14669,"Cosyma, EPU",3,2,"31-10-2020, 31-03-2024","31-10-2020, 31-03-2024"
75,14741,PCH-Fischer GmbH & Co. KG,1,0,NaN,NaN
77,14743,Ingenieurbüro für Bautechnik,1,0,NaN,NaN
90,14757,UnternehmensGrün e.V.,2,1,31-12-2017,31-12-2017
109,14779,WBS TRAINING AG,3,2,"30-11-2018, 31-01-2022, 28-02-2026","31-01-2022, 30-11-2018"
113,14788,Lilly Deutschland GmbH,3,2,"31-01-2022, 31-12-2023, 28-02-2026","31-01-2022, 31-12-2023"
116,14793,Landesverband Amateurtheater Baden-Württemberg...,1,0,NaN,NaN
134,14812,Hans Stockmar GmbH & Co.KG,3,2,"30-04-2022, 31-01-2024, 28-02-2026","30-04-2022, 31-01-2024"
